In [42]:
import os
from pathlib import Path

import polars as pl
import polars.selectors as cs
import pandas as pd
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [2]:
competition_path = Path("/Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/")
mlflow_path = Path(competition_path, "mlruns/")
data_path = Path(competition_path, "data/")

In [9]:
mlflow.set_tracking_uri(mlflow_path)

In [3]:
client = mlflow.tracking.MlflowClient()

In [4]:
client.search_experiments()

[<Experiment: artifact_location='file:///Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/notebooks/mlruns/0', creation_time=1696798233292, experiment_id='0', last_update_time=1696798233292, lifecycle_stage='active', name='Default', tags={}>]

In [34]:
client.tracking_uri

'file:///Users/zacklarsen/Documents/Projects/kaggle-wids-datathon-2020/notebooks/mlruns'

In [57]:
experiment = mlflow.get_experiment_by_name('Default')

Traceback (most recent call last):
  File "/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 303, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 396, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1304, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1297, in _read_helper
    result = read_yaml(root, file_name)
  File "/Users/zacklarsen/mambaforge/envs/wids2020/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 285, in read_yaml
    raise 

In [11]:
mlflow.search_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.device,params.monotone_constraints,params.multi_strategy,params.base_score,...,params.validate_parameters,params.n_jobs,params.subsample,params.max_bin,params.random_state,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.user
0,6183b7ca429c4851b85d7fd6c9743e38,0,FINISHED,/Users/zacklarsen/Documents/Projects/kaggle-wi...,2023-10-08 20:45:54.497000+00:00,2023-10-08 20:45:56.498000+00:00,None,None,None,None,...,None,None,None,None,None,rumbling-eel-607,/Users/zacklarsen/mambaforge/envs/wids2020/lib...,LOCAL,"[{""run_id"": ""6183b7ca429c4851b85d7fd6c9743e38""...",zacklarsen
1,23395a9c824a46cdadc2d5f0943355b5,0,FINISHED,/Users/zacklarsen/Documents/Projects/kaggle-wi...,2023-10-08 20:16:17.328000+00:00,2023-10-08 20:16:18.770000+00:00,None,None,None,None,...,None,None,None,None,None,unruly-loon-873,/Users/zacklarsen/mambaforge/envs/wids2020/lib...,LOCAL,"[{""run_id"": ""23395a9c824a46cdadc2d5f0943355b5""...",zacklarsen
2,1fc461080e3f4961966b20282b07fafa,0,FINISHED,/Users/zacklarsen/Documents/Projects/kaggle-wi...,2023-10-08 20:14:35.493000+00:00,2023-10-08 20:14:37.484000+00:00,None,None,None,None,...,None,None,None,None,None,unleashed-hawk-384,/Users/zacklarsen/mambaforge/envs/wids2020/lib...,LOCAL,"[{""run_id"": ""1fc461080e3f4961966b20282b07fafa""...",zacklarsen


In [12]:
# Get a list of all runs in an experiment
experiment_id = "0"
runs = mlflow.search_runs(experiment_ids=[experiment_id])

In [16]:
runs_pl = pl.DataFrame(runs)

In [17]:
runs_pl

run_id,experiment_id,status,artifact_uri,start_time,end_time,params.device,params.monotone_constraints,params.multi_strategy,params.base_score,params.colsample_bytree,params.reg_alpha,params.early_stopping_rounds,params.max_delta_step,params.colsample_bynode,params.max_cat_to_onehot,params.gamma,params.tree_method,params.maximize,params.colsample_bylevel,params.interaction_constraints,params.min_child_weight,params.verbosity,params.objective,params.verbose_eval,params.learning_rate,params.max_depth,params.grow_policy,params.scale_pos_weight,params.max_leaves,params.num_boost_round,params.custom_metric,params.reg_lambda,params.max_cat_threshold,params.eval_metric,params.sampling_method,params.num_parallel_tree,params.booster,params.validate_parameters,params.n_jobs,params.subsample,params.max_bin,params.random_state,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.user
str,str,str,str,"datetime[ns, UTC]","datetime[ns, UTC]",str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""6183b7ca429c48…","""0""","""FINISHED""","""/Users/zacklar…",2023-10-08 20:45:54.497 UTC,2023-10-08 20:45:56.498 UTC,"""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""binary:logisti…","""True""","""None""","""None""","""None""","""None""","""None""","""100""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""rumbling-eel-6…","""/Users/zacklar…","""LOCAL""","""[{""run_id"": ""6…","""zacklarsen"""
"""23395a9c824a46…","""0""","""FINISHED""","""/Users/zacklar…",2023-10-08 20:16:17.328 UTC,2023-10-08 20:16:18.770 UTC,"""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""binary:logisti…","""True""","""None""","""None""","""None""","""None""","""None""","""100""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""unruly-loon-87…","""/Users/zacklar…","""LOCAL""","""[{""run_id"": ""2…","""zacklarsen"""
"""1fc461080e3f49…","""0""","""FINISHED""","""/Users/zacklar…",2023-10-08 20:14:35.493 UTC,2023-10-08 20:14:37.484 UTC,"""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""binary:logisti…","""True""","""None""","""None""","""None""","""None""","""None""","""100""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""None""","""unleashed-hawk…","""/Users/zacklar…","""LOCAL""","""[{""run_id"": ""1…","""zacklarsen"""


In [23]:
(
    runs_pl
    .select(pl.col("run_id", "experiment_id", "status", "start_time",
                   "end_time", "artifact_uri", "params.objective",
                   "tags.mlflow.runName", "tags.mlflow.log-model.history"))
)

run_id,experiment_id,status,start_time,end_time,artifact_uri,params.objective,tags.mlflow.runName,tags.mlflow.log-model.history
str,str,str,"datetime[ns, UTC]","datetime[ns, UTC]",str,str,str,str
"""6183b7ca429c48…","""0""","""FINISHED""",2023-10-08 20:45:54.497 UTC,2023-10-08 20:45:56.498 UTC,"""/Users/zacklar…","""binary:logisti…","""rumbling-eel-6…","""[{""run_id"": ""6…"
"""23395a9c824a46…","""0""","""FINISHED""",2023-10-08 20:16:17.328 UTC,2023-10-08 20:16:18.770 UTC,"""/Users/zacklar…","""binary:logisti…","""unruly-loon-87…","""[{""run_id"": ""2…"
"""1fc461080e3f49…","""0""","""FINISHED""",2023-10-08 20:14:35.493 UTC,2023-10-08 20:14:37.484 UTC,"""/Users/zacklar…","""binary:logisti…","""unleashed-hawk…","""[{""run_id"": ""1…"


In [24]:
(
    runs_pl
    .select(pl.col("run_id", "experiment_id", "start_time",
                   "end_time", "artifact_uri",
                   "tags.mlflow.runName", "tags.mlflow.log-model.history"))
)

run_id,experiment_id,start_time,end_time,artifact_uri,tags.mlflow.runName,tags.mlflow.log-model.history
str,str,"datetime[ns, UTC]","datetime[ns, UTC]",str,str,str
"""6183b7ca429c48…","""0""",2023-10-08 20:45:54.497 UTC,2023-10-08 20:45:56.498 UTC,"""/Users/zacklar…","""rumbling-eel-6…","""[{""run_id"": ""6…"
"""23395a9c824a46…","""0""",2023-10-08 20:16:17.328 UTC,2023-10-08 20:16:18.770 UTC,"""/Users/zacklar…","""unruly-loon-87…","""[{""run_id"": ""2…"
"""1fc461080e3f49…","""0""",2023-10-08 20:14:35.493 UTC,2023-10-08 20:14:37.484 UTC,"""/Users/zacklar…","""unleashed-hawk…","""[{""run_id"": ""1…"


In [25]:
(
    runs_pl
    .select(pl.col("run_id", "artifact_uri"))
)

run_id,artifact_uri
str,str
"""6183b7ca429c48…","""/Users/zacklar…"
"""23395a9c824a46…","""/Users/zacklar…"
"""1fc461080e3f49…","""/Users/zacklar…"


In [26]:
(
    runs_pl
    .select(pl.col("run_id", "artifact_uri", "tags.mlflow.log-model.history"))
)

run_id,artifact_uri,tags.mlflow.log-model.history
str,str,str
"""6183b7ca429c48…","""/Users/zacklar…","""[{""run_id"": ""6…"
"""23395a9c824a46…","""/Users/zacklar…","""[{""run_id"": ""2…"
"""1fc461080e3f49…","""/Users/zacklar…","""[{""run_id"": ""1…"
